In [ ]:
!pip install -q h5py typing-extensions wheel
!pip install -q -U bitsandbytes
!pip install -q -U fsspec==2025.3.0
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

!pip install -q datasets

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
model_id = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # Activate nested quantization for 4-bit base models (double quantization)
    bnb_4bit_quant_type="nf4", # Quantization type (fp4 or nf4), According to QLoRA paper, for training 4-bit base models (e.g. using LoRA adapters) one should use
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model


from transformers import AutoModelForSeq2SeqLM 
from peft import LoraConfig, AdaLoraModel, AdaLoraConfig

TARGET_MODULES=["q_proj","k_proj", "v_proj", "gate_proj", "down_proj","up_proj"] #['q','k'] 
LORA_R = 8
LORA_ALPHA = 8
LORA_DROPOUT = 0.05
init_lora_weights = 'pissa'
rs = True
dw = True
config = AdaLoraConfig(
    peft_type="ADALORA", task_type="CAUSAL_LM", bias="none",
    init_r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=TARGET_MODULES,
    tinit=100,                            # 初始稳定步数（不调整秩）
    tfinal=1000,                          # 开始动态调整的步数
    deltaT=10,                            # 秩调整间隔步数
    # 
    use_rslora = rs,
    init_lora_weights = init_lora_weights,
    use_dora=dw,
  
)
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model = AdaLoraModel(model, config, "default")

model = get_peft_model(model, config)

model.print_trainable_parameters()



In [ ]:
OUTPUT_DIR = f".{model_id}_rs{rs}_dw{dw}_{init_lora_weights}/checkpoints"

training_arguments = transformers.TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim='paged_adamw_32bit',    # 
    save_steps=0,
    logging_steps=1,
    learning_rate=2e-7,   # 
    weight_decay=0.001,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine", 
    gradient_checkpointing=True,
    report_to="none"
)

    # evaluation_strategy = 'epoch',
    # warmup_steps=50,
    # fp16=True,
    # logging_steps=50,


In [ ]:
from transformers import TrainerCallback

class AdaLoraUpdateCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        # 获取 AdaLora 配置
        adalora_config = self.model.peft_config[self.model.trainable_adapter_name]
        # 每隔 deltaT 步触发一次秩调整
        if state.global_step % adalora_config.deltaT == 0:
            self.model.update_and_allocate(state.global_step)
            print(f"Step {state.global_step}: Rank pattern updated to")
        return control

# 初始化 Trainer 并添加回调
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    callbacks=[AdaLoraUpdateCallback()],  # 关键！
)
trainer.train()


In [ ]:
# 训练循环中需周期性调用秩调整
model.train()

for step, batch in enumerate(train_loader):
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    
    # 每隔 deltaT 步更新秩
    if step > peft_config.tinit and step % peft_config.deltaT == 0:
        model.update_and_allocate(step)  # 动态调整秩

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")